In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import xgboost
import lightgbm as lgb
import catboost
from sklearn.model_selection import KFold

In [2]:
from utility.data_cleaning import *
from utility.data_preprocessing import *
from utility.model_building import *
from config.configuration import *

In [3]:
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [4]:
df=pd.read_excel(input_data_path)
df

,ClaimNumber,LossDate,PolicyNumber,PolicyEntryDate,PolicyExpirationDate,ClaimLossDesc,ClaimStatus,Examiner,ClaimLOB,InsuredName,...,YearBuilt,Stories,Comments,ConstructionTypeDesc,FormattedAddress,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit
0,2003862,2019-04-18 00:00:00.000,5010232,2019-05-03 08:09:04.880,2019-07-01 00:00:00.000,back up causing sewage water in from the drain...,Closed,rgibson,BOP Property,"ETHERIDGE OWNERS, INC.",...,1965,8,ClassCode:65146,MasonryNonCombustible,"3114 WISCONSIN AVENUE, NW\n\rWASHINGTON, DC 20016",20016,7626.78,4758237.0,30800,5090954
1,2004989,2020-01-01 00:00:00.000,7146314,2020-01-02 09:54:03.083,2020-05-04 00:00:00.000,Fire in the kitchen.,Closed,rgibson,BOP Property,"H W Owens, Inc",...,1969,2,ClassCode:65132,JoistedMasonry,"4000-4062 Tangle Drive\n\rRichmond, VA 23228",23228,17492.12,5042916.0,30480,2560320
2,2004989,2020-01-01 00:00:00.000,7146314,2020-01-02 09:54:03.083,2020-05-04 00:00:00.000,Fire in the kitchen.,Closed,rgibson,BOP Property,"H W Owens, Inc",...,1969,2,ClassCode:65132,JoistedMasonry,"4000-4062 Tangle Drive\n\rRichmond, VA 23228",23228,17492.12,5042916.0,30480,2560320
3,2005666,2020-04-30 00:00:00.000,7120072,2020-05-15 12:29:16.837,2020-10-10 00:00:00.000,Façade sign was torn from the building and han...,Closed,ahart,BOP Property,"RLGVS, LLC",...,2012,1,ClassCode:59999,MasonryNonCombustible,"1 NEWBERRY PARKWAY\n\rETTERS, PA 17319",17319,3009.98,3640000.0,35000,5426100
4,2006935,2021-01-27 00:00:00.000,7120072,2021-02-12 13:24:24.330,2021-10-10 00:00:00.000,Fire damage to the building,Closed,rgibson,BOP Property,"RLGVS, LLC",...,2012,1,ClassCode:59999,MasonryNonCombustible,"1 NEWBERRY PARKWAY\n\rETTERS, PA 17319",17319,48695.03,3640000.0,35000,5426100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2011122,2023-03-03 00:00:00.000,7228908,2023-09-26 08:18:36.867,2023-12-05 00:00:00.000,high wind and rain caused roof damage.,Open,ssexton,BOP Property,"PLK Communities, LLC; ETAL",...,1950,1,ClassCode:65146,Frame,"6480-6478 Cheviot Road\n\rCincinnati, OH 45247",45247,NaN,4617938.0,24440,3666000
203,2011122,2023-03-03 00:00:00.000,7228908,2023-09-26 08:18:36.867,2023-12-05 00:00:00.000,high wind and rain caused roof damage.,Open,ssexton,BOP Property,"PLK Communities, LLC; ETAL",...,1950,1,ClassCode:65146,Frame,"6480-6478 Cheviot Road\n\rCincinnati, OH 45247",45247,NaN,4617938.0,24440,3666000
204,2011123,2023-03-03 00:00:00.000,7228908,2023-09-26 08:26:02.633,2023-12-05 00:00:00.000,high winds and rain caused roof damage.,Open,ssexton,BOP Property,"PLK Communities, LLC; ETAL",...,1950,1,ClassCode:65146,Frame,"6458 Cheviot Road\n\rCincinnati, OH 45247",45247,NaN,2231296.8,11280,1692000
205,2011123,2023-03-03 00:00:00.000,7228908,2023-09-26 08:26:02.633,2023-12-05 00:00:00.000,high winds and rain caused roof damage.,Open,ssexton,BOP Property,"PLK Communities, LLC; ETAL",...,1950,1,ClassCode:65146,Frame,"6458 Cheviot Road\n\rCincinnati, OH 45247",45247,NaN,2231296.8,11280,1692000


In [5]:
df['Limit_val'] = df['Limit'].str.extract(r'(\d+)')

In [6]:
df = df[selected_columns]
df_clean= clean_data(df)
df_clean

,Jurisdiction,CoverageType,Deductible,Limit_val,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit
0,dist. of columbia,bop-building,fixeddollar - 2500.00,550637568,water,1965,8,classcode:65146,masonrynoncombustible,20016,7626.78,4758237.0,30800,5090954
1,virginia,bop-building,fixeddollar - 5000.00,2560320,fire,1969,2,classcode:65132,joistedmasonry,23228,17492.12,5042916.0,30480,2560320
2,virginia,bop-building,fixeddollar - 5000.00,1,fire,1969,2,classcode:65132,joistedmasonry,23228,17492.12,5042916.0,30480,2560320
3,pennsylvania,bop-building,fixeddollar - 2500.00,704688296,wind,2012,1,classcode:59999,masonrynoncombustible,17319,3009.98,3640000.0,35000,5426100
4,pennsylvania,bop-building,fixeddollar - 2500.00,732875728,fire,2012,1,classcode:59999,masonrynoncombustible,17319,48695.03,3640000.0,35000,5426100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,ohio,bop-building,fixeddollar - 25000.00,3666000,wind,1950,1,classcode:65146,frame,45247,NaN,4617938.0,24440,3666000
203,ohio,bop-building,fixeddollar - 25000.00,1,wind,1950,1,classcode:65146,frame,45247,NaN,4617938.0,24440,3666000
204,ohio,bop-building,fixeddollar - 25000.00,1692000,wind,1950,1,classcode:65146,frame,45247,NaN,2231296.8,11280,1692000
205,ohio,bop-building,fixeddollar - 25000.00,1,wind,1950,1,classcode:65146,frame,45247,NaN,2231296.8,11280,1692000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Jurisdiction          207 non-null    object 
 1   CoverageType          207 non-null    object 
 2   Deductible            207 non-null    object 
 3   Limit_val             207 non-null    object 
 4   InjuryCause           206 non-null    object 
 5   YearBuilt             207 non-null    int64  
 6   Stories               207 non-null    int64  
 7   Comments              207 non-null    object 
 8   ConstructionTypeDesc  207 non-null    object 
 9   ZipCode               207 non-null    object 
 10  LossAmount            126 non-null    float64
 11  E2Value               207 non-null    float64
 12  SquareFootage         207 non-null    int64  
 13  BuildingLimit         207 non-null    int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 22.8+ KB
          Jurisd

,Jurisdiction,CoverageType,Limit_val,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit,Deductible_type,Deductible_val,YearGap
0,dist. of columbia,bop-building,550637568.0,water,8,classcode:65146,masonrynoncombustible,20016,7626.78,4758237.0,30800,5090954,fixeddollar,2500,58
1,virginia,bop-building,2560320.0,fire,2,classcode:65132,joistedmasonry,23228,17492.12,5042916.0,30480,2560320,fixeddollar,5000,54
2,virginia,bop-building,1.0,fire,2,classcode:65132,joistedmasonry,23228,17492.12,5042916.0,30480,2560320,fixeddollar,5000,54
3,pennsylvania,bop-building,704688296.0,wind,1,classcode:59999,masonrynoncombustible,17319,3009.98,3640000.0,35000,5426100,fixeddollar,2500,11
4,pennsylvania,bop-building,732875728.0,fire,1,classcode:59999,masonrynoncombustible,17319,48695.03,3640000.0,35000,5426100,fixeddollar,2500,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,ohio,bop-building,3666000.0,wind,1,classcode:65146,frame,45247,NaN,4617938.0,24440,3666000,fixeddollar,25000,73
203,ohio,bop-building,1.0,wind,1,classcode:65146,frame,45247,NaN,4617938.0,24440,3666000,fixeddollar,25000,73
204,ohio,bop-building,1692000.0,wind,1,classcode:65146,frame,45247,NaN,2231296.8,11280,1692000,fixeddollar,25000,73
205,ohio,bop-building,1.0,wind,1,classcode:65146,frame,45247,NaN,2231296.8,11280,1692000,fixeddollar,25000,73


In [7]:
X_train, y_train, X_test, y_test= preprocess_and_save(df_clean)

C:\Users\SmrutisikhaJena\Documents\GitHub\Risk_Predictive_Model\utility\data_preprocessing.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LossAmount'] = df['LossAmount'].apply(extract_amount)


,Jurisdiction,CoverageType,Limit_val,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit,Deductible_type,Deductible_val,YearGap
0,1,0,550637568.0,7,8,13,2,59,7626.78,4758237.00,30800,5090954,0,2500,58
1,6,0,2560320.0,1,2,7,1,65,17492.12,5042916.00,30480,2560320,0,5000,54
2,6,0,1.0,1,2,7,1,65,17492.12,5042916.00,30480,2560320,0,5000,54
3,5,0,704688296.0,8,1,4,2,34,3009.98,3640000.00,35000,5426100,0,2500,11
4,5,0,732875728.0,1,1,4,2,34,48695.03,3640000.00,35000,5426100,0,2500,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,5,0,896231.0,5,2,8,1,6,3175.00,896231.52,4104,896231,0,2500,63
312,5,0,1500000.0,7,4,14,0,36,350.00,3895202.50,10000,1500000,0,2500,123
313,4,0,930000.0,1,1,4,1,76,4898.45,1038508.80,6720,930000,0,2500,57
314,1,0,848000.0,7,3,16,0,61,2781.20,848000.00,4000,848000,1,2500,1


In [8]:
train_and_evaluate_model(X_train, y_train, X_test, y_test)

Linear Regression
113282.83851918852
0.12307420483152687
XGBoost
16835.98532816885
0.9806308002783899
RF
49085.96075080091
0.8353547553850617
CATBoost
97023.95810186022
0.3567310697485603
cbr_Kfold
26565.65952890102
0.3435292887582857
xgb_Kfold
9542.053968837212
0.9153049776354928
 rf_model
14832.092394148935
0.7953654342193246
ensemble
15263.337503117642
0.7832928878043945


(49085.96075080091, 0.8353547553850617)